In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5602,2020-11-16T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1501,224,1725,28040,...,958,1461,12296,932,26616.0,16377.0,42993,824760,570817.0,NaN
5603,2020-11-16T17:00:00,ITA,9,Toscana,43.769231,11.255889,1777,284,2061,51790,...,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0,effettuati 45427 tamponi rapidi antigenici di...
5604,2020-11-16T17:00:00,ITA,10,Umbria,43.106758,12.388247,375,72,447,10792,...,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0,si segnala che avendo n.2 ricoverati provenien...
5605,2020-11-16T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,151,16,167,2099,...,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5602,2020-11-16T17:00:00,19,Sicilia,1501,224,1725,28040,29765,958,1461,12296,932,26616.0,16377.0,42993,824760,570817.0
5603,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0
5604,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0
5605,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-16', '2020-11-15')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,50884,5117,53935.0,75006.0,128941,1288923,815862.0
1,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0
2,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,153103,19466,245017.0,79891.0,324908,3568531,2160155.0
3,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,37277,2867,27639.0,75203.0,102842,2571888,993185.0
4,2020-11-16T17:00:00,6,Friuli Venezia Giulia,407,47,454,9601,10055,128,456,8792,546,16657.0,2736.0,19393,623585,254630.0
5,2020-11-16T17:00:00,7,Liguria,1381,111,1492,14409,15901,163,365,25717,2124,30461.0,13281.0,43742,529552,266634.0
6,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,30570,5067,58909.0,32157.0,91066,1873435,980006.0
7,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0
8,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0
9,2020-11-16T17:00:00,11,Marche,499,78,577,13634,14211,162,282,8268,1119,23400.0,198.0,23598,370514,217369.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,49190,5046,52536.0,72929.0,125465,1273021,806208.0
1,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0
2,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,138872,19367,241894.0,78886.0,320780,3550494,2152870.0
3,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,36525,2845,27442.0,73434.0,100876,2562531,989643.0
4,2020-11-15T17:00:00,6,Friuli Venezia Giulia,401,43,444,9483,9927,398,608,8482,528,16278.0,2659.0,18937,620174,253332.0
5,2020-11-15T17:00:00,7,Liguria,1393,115,1508,14230,15738,404,822,25530,2109,30385.0,12992.0,43377,527347,265754.0
6,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,30275,5044,57557.0,30963.0,88520,1858993,972719.0
7,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0
8,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0
9,2020-11-15T17:00:00,11,Marche,483,77,560,13489,14049,630,707,8156,1111,23200.0,116.0,23316,368894,216441.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,50884,5117,53935.0,75006.0,128941,1288923,815862.0
1,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0
2,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,153103,19466,245017.0,79891.0,324908,3568531,2160155.0
3,2020-11-16T17:00:00,4,Trentino-Alto Adige,873,75,948,12458,13406,154,725,16245,944,24646.0,5949.0,30595,609621,NaN
4,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,37277,2867,27639.0,75203.0,102842,2571888,993185.0
5,2020-11-16T17:00:00,6,Friuli Venezia Giulia,407,47,454,9601,10055,128,456,8792,546,16657.0,2736.0,19393,623585,254630.0
6,2020-11-16T17:00:00,7,Liguria,1381,111,1492,14409,15901,163,365,25717,2124,30461.0,13281.0,43742,529552,266634.0
7,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,30570,5067,58909.0,32157.0,91066,1873435,980006.0
8,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0
9,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,49190,5046,52536.0,72929.0,125465,1273021,806208.0
1,1,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0
2,2,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,138872,19367,241894.0,78886.0,320780,3550494,2152870.0
3,19,2020-11-15T17:00:00,4,Trentino-Alto Adige,849,76,925,12327,13252,359,896,15696,922,24011.0,5859.0,29870,605613,NaN
4,3,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,36525,2845,27442.0,73434.0,100876,2562531,989643.0
5,4,2020-11-15T17:00:00,6,Friuli Venezia Giulia,401,43,444,9483,9927,398,608,8482,528,16278.0,2659.0,18937,620174,253332.0
6,5,2020-11-15T17:00:00,7,Liguria,1393,115,1508,14230,15738,404,822,25530,2109,30385.0,12992.0,43377,527347,265754.0
7,6,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,30275,5044,57557.0,30963.0,88520,1858993,972719.0
8,7,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0
9,8,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,50884,5117,53935.0,75006.0,128941,1288923,815862.0,4341375
1,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0,125501
2,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,153103,19466,245017.0,79891.0,324908,3568531,2160155.0,10103969
3,2020-11-16T17:00:00,4,Trentino-Alto Adige,873,75,948,12458,13406,154,725,16245,944,24646.0,5949.0,30595,609621,NaN,1074819
4,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,37277,2867,27639.0,75203.0,102842,2571888,993185.0,4907704
5,2020-11-16T17:00:00,6,Friuli Venezia Giulia,407,47,454,9601,10055,128,456,8792,546,16657.0,2736.0,19393,623585,254630.0,1211357
6,2020-11-16T17:00:00,7,Liguria,1381,111,1492,14409,15901,163,365,25717,2124,30461.0,13281.0,43742,529552,266634.0,1543127
7,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,30570,5067,58909.0,32157.0,91066,1873435,980006.0,4467118
8,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0,3722729
9,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,50884,5117,53935.0,75006.0,128941,1288923,815862.0,4341375,575
1,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0,125501,20
2,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,153103,19466,245017.0,79891.0,324908,3568531,2160155.0,10103969,1036
3,2020-11-16T17:00:00,4,Trentino-Alto Adige,873,75,948,12458,13406,154,725,16245,944,24646.0,5949.0,30595,609621,NaN,1074819,106
4,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,37277,2867,27639.0,75203.0,102842,2571888,993185.0,4907704,1016
5,2020-11-16T17:00:00,6,Friuli Venezia Giulia,407,47,454,9601,10055,128,456,8792,546,16657.0,2736.0,19393,623585,254630.0,1211357,180
6,2020-11-16T17:00:00,7,Liguria,1381,111,1492,14409,15901,163,365,25717,2124,30461.0,13281.0,43742,529552,266634.0,1543127,209
7,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,30570,5067,58909.0,32157.0,91066,1873435,980006.0,4467118,634
8,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0,3722729,523
9,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,...,82.53,-10340,-3932,18037,7285.0,99,14231,22.89,0.04086,3.22
1,2020-11-16T17:00:00,15,Campania,2190,192,2382,89132,91514,3257,4079,...,38.02,3293,308,25110,20653.0,37,785,16.24,0.07050,1.99
2,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,...,65.74,1552,-206,15902,9654.0,71,1694,21.86,0.08007,2.97
3,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,...,38.96,2119,-275,14442,7287.0,23,295,17.64,0.05702,2.04
4,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,...,54.30,1490,-220,15527,6513.0,36,854,15.67,0.06536,2.20
5,2020-11-16T17:00:00,12,Lazio,3099,278,3377,65893,69270,1961,2407,...,32.82,2017,-205,19699,13538.0,34,412,12.22,0.04104,1.48
6,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,...,24.80,1130,-826,9357,3542.0,22,752,21.01,0.04006,2.10
7,2020-11-16T17:00:00,19,Sicilia,1501,224,1725,28040,29765,958,1461,...,38.10,926,39,8151,5564.0,36,467,17.92,0.02941,0.87
8,2020-11-16T17:00:00,16,Puglia,1295,182,1477,24048,25525,823,1044,...,49.32,787,139,4425,2381.0,36,185,23.59,0.02605,0.89
9,2020-11-16T17:00:00,4,Trentino-Alto Adige,873,75,948,12458,13406,154,725,...,70.75,131,-171,4008,NaN,22,549,18.09,0.06745,2.85


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,855,181,1036,82.53
1,Valle d'Aosta,16,4,20,80.00
2,Trentino-Alto Adige,75,31,106,70.75
3,Piemonte,378,197,575,65.74
4,Umbria,72,45,117,61.54
5,Marche,78,65,143,54.55
6,Toscana,284,239,523,54.30
7,Liguria,111,98,209,53.11
8,Puglia,182,187,369,49.32
9,Abruzzo,59,89,148,39.86


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
